# MLOps End to End Workflow (III)

Implementation of an end-to-end ML Ops workflow for the use case to detect fraudulent credit card transactions, see [Kaggle dataset](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud).

This set of notebooks cover:

[Experimentation](01-experimentation.ipynb):
1. Set up: Creation of the Vertex Dataset, extraction of the schema
2. Implementation of a TFX pipeline

[CICD](02-cicd.ipynb):

3. Deployment of the Vertex AI Pipeline through a CI/CD process
4. Deployment of a Continuous Training pipeline that can be triggered via Pub/Sub and produces a model in the Model Registry
5. Deployment of the Inference Pipeline consisting of a Cloud Function that retrieves features from Feature Store and calls the model on a Vertex AI Endpoint
6. Deployment of the model to a Vertex AI Endpoint through a CI/CD process.

[Prediction](03-prediction.ipynb):

7. Deploy the model to an endpoint
8. Create a test prediction


### Configuration

In [1]:
#%load_ext autoreload
#%autoreload 2

import os
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv
from google.cloud import bigquery
import matplotlib.pyplot as plt

from google.cloud import aiplatform as vertex_ai

import yaml
import os

with open('mainconfig.yaml') as f:
    main_config = yaml.safe_load(f)

# select your config    
main_config = main_config['creditcards']

PROJECT = main_config['project'] 
REGION = main_config['region'] 
DOCKER_REPO = main_config['docker_repo']

SERVICE_ACCOUNT = main_config['service_account']

# BigQuery and data locations

BQ_SOURCE_TABLE= main_config['bq']['source_table'] # raw input
ML_TABLE = main_config['bq']['ml_table'] # the one we will use for the training

BQ_DATASET_NAME = main_config['bq']['dataset']
BQ_LOCATION = main_config['bq']['location'] # multiregion provides more resilience

VERTEX_DATASET_NAME = main_config['vertex_dataset_name']

RAW_SCHEMA_DIR = main_config['raw_schema_dir']

BUCKET =  main_config['bucket']

# TFX and model config

# model version
VERSION = main_config['version']


MODEL_DISPLAY_NAME = f'{VERTEX_DATASET_NAME}-classifier-{VERSION}'
WORKSPACE = f'gs://{BUCKET}/{VERTEX_DATASET_NAME}'

MLMD_SQLLITE = 'mlmd.sqllite'
ARTIFACT_STORE = os.path.join(WORKSPACE, 'tfx_artifacts_interactive')
MODEL_REGISTRY = os.path.join(WORKSPACE, 'model_registry')
PIPELINE_NAME = f'{MODEL_DISPLAY_NAME}-train-pipeline'
PIPELINE_ROOT = os.path.join(ARTIFACT_STORE, PIPELINE_NAME)

ENDPOINT_DISPLAY_NAME = f'{VERTEX_DATASET_NAME}-classifier'

FEATURESTORE_ID = main_config['featurestore_id']

CF_REGION = main_config['cloudfunction_region']

DATAFLOW_SUBNETWORK = f"https://www.googleapis.com/compute/v1/projects/{PROJECT}/regions/{REGION}/subnetworks/{main_config['dataflow']['subnet']}"
DATAFLOW_SERVICE_ACCOUNT = main_config['dataflow']['service_account']

CLOUDBUILD_SA = f'projects/{PROJECT}/serviceAccounts/{SERVICE_ACCOUNT}'

LIMIT=main_config['limit']

print("Project ID:", PROJECT)
print("Region:", REGION)
print("Service Account:", SERVICE_ACCOUNT)

vertex_ai.init(
    project=PROJECT,
    location=REGION
)

Project ID: vertex-mlops-new
Region: us-central1
Service Account: 122302747547-compute@developer.gserviceaccount.com


In [2]:
os.environ["VERTEX_DATASET_NAME"] = VERTEX_DATASET_NAME
os.environ["MODEL_DISPLAY_NAME"] = MODEL_DISPLAY_NAME
os.environ["PIPELINE_NAME"] = PIPELINE_NAME
os.environ["PROJECT"] = PROJECT
os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT
os.environ["REGION"] = REGION
os.environ["GCS_LOCATION"] = f"gs://{BUCKET}/{VERTEX_DATASET_NAME}"
os.environ["SERVICE_ACCOUNT"] = DATAFLOW_SERVICE_ACCOUNT
os.environ["GCS_LOCATION"] = f"gs://{BUCKET}/{VERTEX_DATASET_NAME}/e2e_tests"
os.environ["SUBNETWORK"] = DATAFLOW_SUBNETWORK

In [ ]:
!python -m pytest src/tests/pipeline_deployment_tests.py::test_e2e_pipeline -s

In [ ]:
config.BEAM_RUNNER

In [ ]:
from src.tfx_pipelines import config
import importlib

importlib.reload(config)

for key, value in config.__dict__.items():
    if key.isupper(): print(f'{key}: {value}')

In [ ]:
# Repo should has been created in the Terraform automation stage
#! gcloud artifacts repositories create {VERTEX_DATASET_NAME} --location={REGION} --repository-format=docker

In [ ]:
# You can also use build/Dockerfile.dataflow in case Internet access is not allowed
!cp build/Dockerfile.dataflow_internet Dockerfile

In [ ]:
os.environ["DOCKER_REPO"] = f"{DOCKER_REPO}/vertex:latest"
!gcloud builds submit --project=$PROJECT --billing-project=$PROJECT --region $REGION --tag $DOCKER_REPO/dataflow:latest . --timeout=15m --machine-type=e2-highcpu-8 --suppress-logs

In [ ]:
!echo $TFX_IMAGE_URI

In [ ]:
!echo $PYTHONPATH

In [ ]:
!cp build/Dockerfile.vertex_internet Dockerfile
!gcloud builds submit --project=$PROJECT --billing-project=$PROJECT --region $REGION --tag $TFX_IMAGE_URI . --timeout=15m --machine-type=e2-highcpu-8 --suppress-logs

In [ ]:
PIPELINES_STORE = f"gs://{BUCKET}/{VERTEX_DATASET_NAME}/compiled_pipelines/"
!gsutil cp {pipeline_definition_file} {PIPELINES_STORE}
PIPELINES_STORE

In [ ]:
from src.tfx_pipelines import config, runner

pipeline_definition_file = f'{config.PIPELINE_NAME}.json'
pipeline_definition = runner.compile_training_pipeline(pipeline_definition_file)

In [ ]:
from google.cloud.aiplatform import pipeline_jobs
    
job = pipeline_jobs.PipelineJob(template_path = pipeline_definition_file,
                                display_name=VERTEX_DATASET_NAME,
                                #enable_caching=False,
                                parameter_values={
                                    'learning_rate': 0.003,
                                    'batch_size': 512,
                                    'steps_per_epoch': int(config.TRAIN_LIMIT) // 512,
                                    'hidden_units': '128,128',
                                    'num_epochs': 30,
                                })

job.run(sync=False, service_account=DATAFLOW_SERVICE_ACCOUNT)

In [6]:
CICD_IMAGE_URI = f"{DOCKER_REPO}/cicd:latest"
CICD_IMAGE_URI

'us-central1-docker.pkg.dev/vertex-mlops-new/creditcards/cicd:latest'

In [ ]:
# For the CICD container, we are just adding the build/* dir
!cp build/Dockerfile.cicd_internet build/Dockerfile
!gcloud builds submit --project=$PROJECT --billing-project=$PROJECT --region $REGION --tag $CICD_IMAGE_URI build/. --timeout=15m --machine-type=e2-highcpu-8 --suppress-logs

In [7]:
REPO_URL = main_config['git']['repo_url']
BRANCH = main_config['git']['branch']


GCS_LOCATION = f"gs://{BUCKET}/{VERTEX_DATASET_NAME}/"
TEST_GCS_LOCATION = f"gs://{BUCKET}/{VERTEX_DATASET_NAME}/e2e_tests"
CI_TRAIN_LIMIT = 1000
CI_TEST_LIMIT = 100
CI_UPLOAD_MODEL = 0
CI_ACCURACY_THRESHOLD = -0.1 # again setting accuracy threshold to negative
BEAM_RUNNER = "DataflowRunner"
TRAINING_RUNNER = "vertex"
VERSION = 'latest'
PIPELINE_NAME = f'{MODEL_DISPLAY_NAME}-train-pipeline'
PIPELINES_STORE = f"{GCS_LOCATION}compiled_pipelines/"

TFX_IMAGE_URI = f"{DOCKER_REPO}/vertex:{VERSION}"
DATAFLOW_IMAGE_URI = f"{DOCKER_REPO}/dataflow:latest"

REPO_NAME = REPO_URL.split('/')[-1]
DESCR=f'"Deploy train pipeline to GCS from {BRANCH}"'


SUBSTITUTIONS=f"""\
_REPO_URL='{REPO_URL}',\
_BRANCH={BRANCH},\
_CICD_IMAGE_URI={CICD_IMAGE_URI},\
_PROJECT={PROJECT},\
_REGION={REGION},\
_GCS_LOCATION={GCS_LOCATION},\
_TEST_GCS_LOCATION={TEST_GCS_LOCATION},\
_BQ_LOCATION={BQ_LOCATION},\
_BQ_DATASET_NAME={BQ_DATASET_NAME},\
_ML_TABLE={ML_TABLE},\
_VERTEX_DATASET_NAME={VERTEX_DATASET_NAME},\
_MODEL_DISPLAY_NAME={MODEL_DISPLAY_NAME},\
_CI_TRAIN_LIMIT={CI_TRAIN_LIMIT},\
_CI_TEST_LIMIT={CI_TEST_LIMIT},\
_CI_UPLOAD_MODEL={CI_UPLOAD_MODEL},\
_CI_ACCURACY_THRESHOLD={CI_ACCURACY_THRESHOLD},\
_BEAM_RUNNER={BEAM_RUNNER},\
_TRAINING_RUNNER={TRAINING_RUNNER},\
_DATAFLOW_IMAGE_URI={DATAFLOW_IMAGE_URI},\
_TFX_IMAGE_URI={TFX_IMAGE_URI},\
_PIPELINE_NAME={PIPELINE_NAME},\
_PIPELINES_STORE={PIPELINES_STORE},\
_SUBNETWORK={DATAFLOW_SUBNETWORK},\
_GCS_BUCKET={BUCKET}/cloudbuild,\
_SERVICE_ACCOUNT={DATAFLOW_SERVICE_ACCOUNT},\
_WORKDIR={REPO_NAME}\
"""
!echo $SUBSTITUTIONS

_REPO_URL=git@github.com:bluemetrica/final_gcp.git,_BRANCH=main,_CICD_IMAGE_URI=us-central1-docker.pkg.dev/vertex-mlops-new/creditcards/cicd:latest,_PROJECT=vertex-mlops-new,_REGION=us-central1,_GCS_LOCATION=gs://vertex-mlops-new/creditcards/,_TEST_GCS_LOCATION=gs://vertex-mlops-new/creditcards/e2e_tests,_BQ_LOCATION=us-central1,_BQ_DATASET_NAME=creditcards,_ML_TABLE=creditcards_ml,_VERTEX_DATASET_NAME=creditcards,_MODEL_DISPLAY_NAME=creditcards-classifier-v02,_CI_TRAIN_LIMIT=1000,_CI_TEST_LIMIT=100,_CI_UPLOAD_MODEL=0,_CI_ACCURACY_THRESHOLD=-0.1,_BEAM_RUNNER=DataflowRunner,_TRAINING_RUNNER=vertex,_DATAFLOW_IMAGE_URI=us-central1-docker.pkg.dev/vertex-mlops-new/creditcards/dataflow:latest,_TFX_IMAGE_URI=us-central1-docker.pkg.dev/vertex-mlops-new/creditcards/vertex:latest,_PIPELINE_NAME=creditcards-classifier-v02-train-pipeline,_PIPELINES_STORE=gs://vertex-mlops-new/creditcards/compiled_pipelines/,_SUBNETWORK=https://www.googleapis.com/compute/v1/projects/vertex-mlops-new/regions/us-cent

In [ ]:
!gcloud builds submit build/known_hosts.github.zip --config build/pipeline-deployment.yaml --substitutions {SUBSTITUTIONS} --project=$PROJECT --billing-project=$PROJECT --region $REGION --suppress-logs

In [9]:
!echo gcloud beta builds triggers create cloud-source-repositories --repo={REPO_NAME} --branch-pattern=^{BRANCH}$ --description={DESCR} --build-config=mlops-creditcard/build/pipeline-deployment.yaml --substitutions={SUBSTITUTIONS} --billing-project={PROJECT}  --service-account={TRIGGER_SA}

gcloud beta builds triggers create cloud-source-repositories --repo={REPO_NAME} --branch-pattern=^{BRANCH}$ --description={DESCR} --build-config=mlops-creditcard/build/pipeline-deployment.yaml --substitutions={SUBSTITUTIONS} --billing-project={PROJECT} --service-account={TRIGGER_SA}


In [8]:
PUBSUB_TOPIC = f'trigger-{PIPELINE_NAME}'
CLOUD_FUNCTION_NAME = f'trigger-{PIPELINE_NAME}-fn'
GCS_PIPELINE_FILE_LOCATION = os.path.join(PIPELINES_STORE, f'{PIPELINE_NAME}.json')

#### Create Pub/Sub Topic

In [ ]:
!gcloud pubsub topics create {PUBSUB_TOPIC}

#### Deploy Cloud Function

In [10]:
ENV_VARS=f"""\
PROJECT={PROJECT},\
REGION={REGION},\
GCS_PIPELINE_FILE_LOCATION={GCS_PIPELINE_FILE_LOCATION},\
SERVICE_ACCOUNT={SERVICE_ACCOUNT},\
PIPELINE_NAME={PIPELINE_NAME}
"""

!echo {ENV_VARS}

PROJECT=vertex-mlops-new,REGION=us-central1,GCS_PIPELINE_FILE_LOCATION=gs://vertex-mlops-new/creditcards/compiled_pipelines/creditcards-classifier-v02-train-pipeline.json,SERVICE_ACCOUNT=122302747547-compute@developer.gserviceaccount.com,PIPELINE_NAME=creditcards-classifier-v02-train-pipeline


In [ ]:
!rm -rf src/pipeline_triggering/.ipynb_checkpoints

In [ ]:
!gcloud functions deploy {CLOUD_FUNCTION_NAME} --gen2 \
    --region={CF_REGION} \
    --trigger-topic={PUBSUB_TOPIC} \
    --runtime=python38 \
    --source=src/pipeline_triggering\
    --entry-point=trigger_pipeline\
    --stage-bucket={BUCKET}\
    --ingress-settings=internal-only\
    --service-account={SERVICE_ACCOUNT}\
    --update-env-vars={ENV_VARS} 

#### Test triggering the pipeline with a Pub/Sub message

In [12]:
from google.cloud import pubsub
import json

publish_client = pubsub.PublisherClient()
topic = f'projects/{PROJECT}/topics/{PUBSUB_TOPIC}'
data = {
    'num_epochs': 7,
    'learning_rate': 0.0015,
    'batch_size': 512,
    'hidden_units': '256,126'
}
message = json.dumps(data)

_ = publish_client.publish(topic, message.encode())

Check the console to see that it's running.

We now have:

* A pipeline that can be run to test and deploy new training pipelines
* A triggering mechanism to programmatically trigger new training runs
* A training run finish with a new model in the Vertex AI Model Registry


## Deploy the model

Preparation:

* Create a Vertex AI Endpoint
* Create a Vertex AI Feature Store and upload some feature data
* Deploy a Cloud Function that receives prediction requests, pull features from the Feature Store, call the Endpoint and returns a prediction

After this, we will deploy a model deployment pipeline, that will:

* Test the model locally
* Create the Endpoint if necessary
* Deploy the model to the Endpoint
* Test the model on the Endpoint


### Preparation

#### Vertex AI Endpoint

In [13]:
ENDPOINT_DISPLAY_NAME

'creditcards-classifier'

In [14]:
from build.utils import create_endpoint

endpoint = create_endpoint(PROJECT, REGION, ENDPOINT_DISPLAY_NAME)
endpoint

resource name: projects/122302747547/locations/us-central1/endpoints/8324580654775795712

### Model Deployment Pipeline

#### Run the model artifact testing

Artifact testing requires that the model is deployed to the Vertex AI Model Registry.

In [15]:
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["SERVICE_ACCOUNT"] = SERVICE_ACCOUNT
os.environ['ENDPOINT_DISPLAY_NAME'] = ENDPOINT_DISPLAY_NAME
os.environ["MODEL_DISPLAY_NAME"] = MODEL_DISPLAY_NAME

In [ ]:
!python -m pytest src/tests/model_deployment_tests.py::test_model_artifact -s

#### Deploy Model to Endpoint

In [ ]:
!python build/utils.py \
    --mode=deploy-model\
    --project={PROJECT}\
    --region={REGION}\
    --endpoint-display-name={ENDPOINT_DISPLAY_NAME}\
    --model-display-name={MODEL_DISPLAY_NAME}

#### Test model on Endpoint

In [ ]:
!python -m pytest src/tests/model_deployment_tests.py::test_model_endpoint

#### Run the pipeline

In [ ]:
REPO_URL = main_config['git']['repo_url']
BRANCH = main_config['git']['branch']
REPO_NAME = REPO_URL.split('/')[-1]
CICD_IMAGE_URI = f"{DOCKER_REPO}/cicd:latest"

In [ ]:
SUBSTITUTIONS=f"""\
_REPO_URL='{REPO_URL}',\
_BRANCH={BRANCH},\
_CICD_IMAGE_URI={CICD_IMAGE_URI},\
_PROJECT={PROJECT},\
_REGION={REGION},\
_MODEL_DISPLAY_NAME={MODEL_DISPLAY_NAME},\
_ENDPOINT_DISPLAY_NAME={ENDPOINT_DISPLAY_NAME},\
_GCS_BUCKET={BUCKET}/cloudbuild,\
_SERVICE_ACCOUNT={SERVICE_ACCOUNT},\
_WORKDIR={REPO_NAME}\
"""

SUBSTITUTIONS

### Test the build and define a manual trigger

In [ ]:
!echo gcloud builds submit --no-source --config build/model-deployment.yaml --substitutions {SUBSTITUTIONS} --billing-project {PROJECT} --suppress-logs --async

In [16]:
DESCR=f'"Deploy model from branch {BRANCH}"'

In [17]:
!echo gcloud alpha builds triggers create manual --repo={REPO_URL} --repo-type=CLOUD_SOURCE_REPOSITORIES --branch={BRANCH} --description={DESCR} --build-config=mlops-creditcard/build/model-deployment.yaml --substitutions={SUBSTITUTIONS} --billing-project={PROJECT} --service-account={CLOUDBUILD_SA}

gcloud alpha builds triggers create manual --repo=git@github.com:bluemetrica/final_gcp.git --repo-type=CLOUD_SOURCE_REPOSITORIES --branch=main --description=Deploy model from branch main --build-config=mlops-creditcard/build/model-deployment.yaml --substitutions=_REPO_URL=git@github.com:bluemetrica/final_gcp.git,_BRANCH=main,_CICD_IMAGE_URI=us-central1-docker.pkg.dev/vertex-mlops-new/creditcards/cicd:latest,_PROJECT=vertex-mlops-new,_REGION=us-central1,_GCS_LOCATION=gs://vertex-mlops-new/creditcards/,_TEST_GCS_LOCATION=gs://vertex-mlops-new/creditcards/e2e_tests,_BQ_LOCATION=us-central1,_BQ_DATASET_NAME=creditcards,_ML_TABLE=creditcards_ml,_VERTEX_DATASET_NAME=creditcards,_MODEL_DISPLAY_NAME=creditcards-classifier-v02,_CI_TRAIN_LIMIT=1000,_CI_TEST_LIMIT=100,_CI_UPLOAD_MODEL=0,_CI_ACCURACY_THRESHOLD=-0.1,_BEAM_RUNNER=DataflowRunner,_TRAINING_RUNNER=vertex,_DATAFLOW_IMAGE_URI=us-central1-docker.pkg.dev/vertex-mlops-new/creditcards/dataflow:latest,_TFX_IMAGE_URI=us-central1-docker.pkg.dev

## Deploy Prediction Cloud Function

The Cloud Function that performs the final prediction has to:
    
* Receive the features from the prediction request: `V1`, `V2`, ..., `V26`, `Amount`, `userid`
* Use the `userid` to retrieve the features `V27` and `V28` from the Feature Store
* Query the model on the Vertex AI Endpint
* Return the prediction


In [18]:
endpoints = vertex_ai.Endpoint.list(
    filter=f'display_name={ENDPOINT_DISPLAY_NAME}',
    order_by="update_time"
)

if len(endpoints) == 0:
    print(f'No endpoints found with name {ENDPOINT_DISPLAY_NAME}')
endpoint = endpoints[-1]

os.environ['ENDPOINT_NAME'] = endpoint.name

entity = 'user'
os.environ['ENTITY'] = entity
os.environ['FEATURESTORE_ID'] = FEATURESTORE_ID

PREDICT_CLOUD_FUNCTION_NAME = "predict-" + PIPELINE_NAME + "-fn"
PREDICT_CLOUD_FUNCTION_NAME

'predict-creditcards-classifier-v02-train-pipeline-fn'

In [19]:
from src.tests.model_deployment_tests import test_instance
test_instance

{'V1': [-0.906611],
 'V2': [-0.906611],
 'V3': [-0.906611],
 'V4': [-0.906611],
 'V5': [-0.906611],
 'V6': [-0.906611],
 'V7': [-0.906611],
 'V8': [-0.906611],
 'V9': [-0.906611],
 'V10': [-0.906611],
 'V11': [-0.906611],
 'V12': [-0.906611],
 'V13': [-0.906611],
 'V14': [-0.906611],
 'V15': [-0.906611],
 'V16': [-0.906611],
 'V17': [-0.906611],
 'V18': [-0.906611],
 'V19': [-0.906611],
 'V20': [-0.906611],
 'V21': [-0.906611],
 'V22': [-0.906611],
 'V23': [-0.906611],
 'V24': [-0.906611],
 'V25': [-0.906611],
 'V26': [-0.906611],
 'V27': [-0.906611],
 'V28': [-0.906611],
 'Amount': [15.99]}

In [20]:
test_instances = [test_instance]
predictions = endpoint.predict(test_instances).predictions

for prediction in predictions:
    print(prediction)


DEBUG:google.auth.transport.requests:Making request: GET http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true


Making request: GET http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true


DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): metadata.google.internal:80


Starting new HTTP connection (1): metadata.google.internal:80


DEBUG:urllib3.connectionpool:http://metadata.google.internal:80 "GET /computeMetadata/v1/instance/service-accounts/default/?recursive=true HTTP/1.1" 200 193


http://metadata.google.internal:80 "GET /computeMetadata/v1/instance/service-accounts/default/?recursive=true HTTP/1.1" 200 193


DEBUG:google.auth.transport.requests:Making request: GET http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/122302747547-compute@developer.gserviceaccount.com/token?scopes=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform


Making request: GET http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/122302747547-compute@developer.gserviceaccount.com/token?scopes=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform


DEBUG:urllib3.connectionpool:http://metadata.google.internal:80 "GET /computeMetadata/v1/instance/service-accounts/122302747547-compute@developer.gserviceaccount.com/token?scopes=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform HTTP/1.1" 200 1083


http://metadata.google.internal:80 "GET /computeMetadata/v1/instance/service-accounts/122302747547-compute@developer.gserviceaccount.com/token?scopes=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform HTTP/1.1" 200 1083
{'scores': [0.50000155, 0.49999845], 'classes': ['legit', 'fraudulent']}


In [ ]:
#In case model is deployed with explanations
explanations = endpoint.explain(test_instances).explanations

for explanation in explanations:
    print(explanation)


In [21]:
import json

request = {'instances': test_instances}

REQ_JSON=json.dumps(request)
REQ_JSON

'{"instances": [{"V1": [-0.906611], "V2": [-0.906611], "V3": [-0.906611], "V4": [-0.906611], "V5": [-0.906611], "V6": [-0.906611], "V7": [-0.906611], "V8": [-0.906611], "V9": [-0.906611], "V10": [-0.906611], "V11": [-0.906611], "V12": [-0.906611], "V13": [-0.906611], "V14": [-0.906611], "V15": [-0.906611], "V16": [-0.906611], "V17": [-0.906611], "V18": [-0.906611], "V19": [-0.906611], "V20": [-0.906611], "V21": [-0.906611], "V22": [-0.906611], "V23": [-0.906611], "V24": [-0.906611], "V25": [-0.906611], "V26": [-0.906611], "V27": [-0.906611], "V28": [-0.906611], "Amount": [15.99]}]}'

In [22]:
os.environ['PROJECT_ID'] = PROJECT
os.environ['REGION'] = REGION
os.environ['ENDPOINT_ID'] = endpoint.name
os.environ['INPUT_DATA_FILE'] = "INPUT-JSON"
os.environ['REQ_JSON'] = REQ_JSON
!echo ${REQ_JSON} > ${INPUT_DATA_FILE}
!curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)"  -H "Content-Type: application/json"  https://${REGION}-aiplatform.googleapis.com/v1/projects/${PROJECT_ID}/locations/europe-west4/endpoints/${ENDPOINT_ID}:predict  -d "@${INPUT_DATA_FILE}"

{
  "predictions": [
    {
      "classes": [
        "legit",
        "fraudulent"
      ],
      "scores": [
        0.50000155,
        0.49999845
      ]
    }
  ],
  "deployedModelId": "3155886443763597312",
  "model": "projects/122302747547/locations/us-central1/models/4386433469291429888",
  "modelDisplayName": "creditcards-classifier-v02",
  "modelVersionId": "1"
}


## Feature Store (Optional)

### Create Feature Store

In [29]:
from google.cloud.aiplatform_v1beta1 import FeaturestoreOnlineServingServiceClient, FeaturestoreServiceClient

In [30]:
from src.feature_store import feature_store as fs

In [ ]:
fs.create_fs(PROJECT, REGION, FEATURESTORE_ID, "Feature Store for credit card use case")

In [24]:
from google.api_core import operations_v1
from google.cloud.aiplatform_v1beta1 import FeaturestoreOnlineServingServiceClient, FeaturestoreServiceClient, FeatureSelector
from google.cloud.aiplatform_v1beta1.types import featurestore_online_service as featurestore_online_service_pb2
from google.cloud.aiplatform_v1beta1.types import entity_type as entity_type_pb2
from google.cloud.aiplatform_v1beta1.types import feature as feature_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore as featurestore_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore_service as featurestore_service_pb2
from google.cloud.aiplatform_v1beta1.types import io as io_pb2
from google.cloud.aiplatform_v1beta1.types import ListFeaturestoresRequest, CreateFeaturestoreRequest, Featurestore, ListEntityTypesRequest

from google.protobuf.timestamp_pb2 import Timestamp
from google.cloud.aiplatform_v1beta1.types import featurestore_monitoring as featurestore_monitoring_pb2
from google.protobuf.duration_pb2 import Duration


API_ENDPOINT = f"{REGION}-aiplatform.googleapis.com"  
admin_client = FeaturestoreServiceClient(client_options={"api_endpoint": API_ENDPOINT})
parent = f'{admin_client.common_location_path(PROJECT, REGION)}/featurestores/{FEATURESTORE_ID}'
request = ListEntityTypesRequest(parent=parent)

# Make the request
page_result = admin_client.list_entity_types(request=request)

# Handle the response
[x.name.split('/')[-1] for x in page_result]


DEBUG:google.auth._default:Checking None for explicit credentials as part of auth process...


Checking None for explicit credentials as part of auth process...


DEBUG:google.auth._default:Checking Cloud SDK credentials as part of auth process...


Checking Cloud SDK credentials as part of auth process...


DEBUG:google.auth._default:Cloud SDK credentials not found on disk; not using them


...

DEBUG:google.auth.transport.requests:Making request: GET http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/122302747547-compute@developer.gserviceaccount.com/token?scopes=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform


Making request: GET http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/122302747547-compute@developer.gserviceaccount.com/token?scopes=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform


DEBUG:urllib3.connectionpool:http://metadata.google.internal:80 "GET /computeMetadata/v1/instance/service-accounts/122302747547-compute@developer.gserviceaccount.com/token?scopes=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform HTTP/1.1" 200 1083


http://metadata.google.internal:80 "GET /computeMetadata/v1/instance/service-accounts/122302747547-compute@developer.gserviceaccount.com/token?scopes=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform HTTP/1.1" 200 1083


['user']

In [25]:
admin_client.featurestore_path(PROJECT, REGION, FEATURESTORE_ID)

'projects/vertex-mlops-new/locations/us-central1/featurestores/creditcards'

#### Create an entity with features, generate some data and upload it

In [26]:
entity = 'user'
entity_descr = 'User ID'
features = ['v27', 'v28']

In [27]:
fs.create_entity(PROJECT, REGION, FEATURESTORE_ID, entity, entity_descr, features)

NameError: name 'fs' is not defined

In [31]:
import random

filename = f'features_{entity}.csv'

with open(filename, 'w') as f:
    line = f'{entity},{",".join(features)}\n'
    f.write(line)
    for i in range(100):
        f.write(f'user{i},{random.random()},{random.random()}\n')

In [32]:
!tail -20 {filename}

user80,0.20778739654930622,0.5752678220264608
user81,0.4722169790566131,0.35012520802085256
user82,0.7900368926296649,0.7859552450474113
user83,0.17444219766404112,0.9240157714642742
user84,0.21794351815080137,0.6878574235669507
user85,0.1955520890545802,0.8358545966960552
user86,0.33379390711648327,0.14401257156504743
user87,0.6650206053551966,0.15121729300230002
user88,0.1412773624666419,0.5383116454394479
user89,0.6863582211699236,0.8200256330064112
user90,0.034507293487492374,0.18795443783234533
user91,0.2610936676003942,0.6924395077561596
user92,0.25734527101959936,0.027937678117044284
user93,0.31469863584488833,0.2584776413338492
user94,0.08076661048563849,0.7764593930305413
user95,0.8495755936826067,0.6717554652172218
user96,0.4934202525386211,0.7208328100435277
user97,0.9169675563848182,0.6106995792065544
user98,0.7265909354815361,0.850251440541875
user99,0.3996650075157515,0.34054126925472117


In [33]:
BUCKET

'vertex-mlops-new'

In [ ]:
!gsutil cp {filename} gs://{BUCKET}/{filename} 

In [ ]:
gcs_uris = [f'gs://{BUCKET}/{filename}']

fs.ingest_entities_csv(PROJECT, REGION, FEATURESTORE_ID, entity, features, gcs_uris)

Test reading some features back

In [34]:
features_data = {}
for i in range(90,102):
    entity_id = f'user{i}'
    features_data[entity_id] = fs.read_features(PROJECT, REGION, FEATURESTORE_ID, entity, features, entity_id)

features_data

DEBUG:google.auth._default:Checking None for explicit credentials as part of auth process...


Checking None for explicit credentials as part of auth process...


DEBUG:google.auth._default:Checking Cloud SDK credentials as part of auth process...


Checking Cloud SDK credentials as part of auth process...


DEBUG:google.auth._default:Cloud SDK credentials not found on disk; not using them


...

DEBUG:google.auth.transport.requests:Making request: GET http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/122302747547-compute@developer.gserviceaccount.com/token?scopes=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform


Making request: GET http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/122302747547-compute@developer.gserviceaccount.com/token?scopes=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform


DEBUG:urllib3.connectionpool:http://metadata.google.internal:80 "GET /computeMetadata/v1/instance/service-accounts/122302747547-compute@developer.gserviceaccount.com/token?scopes=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform HTTP/1.1" 200 1083


http://metadata.google.internal:80 "GET /computeMetadata/v1/instance/service-accounts/122302747547-compute@developer.gserviceaccount.com/token?scopes=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform HTTP/1.1" 200 1083


{'user90': {'v27': 0.51370059078422, 'v28': 0.21421453329582296},
 'user91': {'v27': 0.003910660120753695, 'v28': 0.4074832126204948},
 'user92': {'v27': 0.4077819162485974, 'v28': 0.8095300774361845},
 'user93': {'v27': 0.14227157871404128, 'v28': 0.45360963152018097},
 'user94': {'v27': 0.8127022395090265, 'v28': 0.3610649548309922},
 'user95': {'v27': 0.568409696583998, 'v28': 0.3408997256732825},
 'user96': {'v27': 0.14994819428359096, 'v28': 0.5677369292043252},
 'user97': {'v27': 0.804031925430529, 'v28': 0.40494853356058313},
 'user98': {'v27': 0.9540305345470473, 'v28': 0.9675106288113674},
 'user99': {'v27': 0.30816454100032054, 'v28': 0.4322800651635872},
 'user100': {},
 'user101': {}}

### Deploy Prediction Cloud Function to use with Feature Store

The Cloud Function that performs the final prediction has to:
    
* Receive the features from the prediction request: `V1`, `V2`, ..., `V26`, `Amount`, `userid`
* Use the `userid` to retrieve the features `V27` and `V28` from the Feature Store
* Query the model on the Vertex AI Endpint
* Return the prediction

#### Test the enpoint with Feature store

In [35]:
from src.tests.model_deployment_tests import test_instance

import base64

if 'V27' in test_instance:
    del test_instance['V27']
if 'V28' in test_instance:
    del test_instance['V28']
test_instance['userid'] = 'user99'

test_instance

{'V1': [-0.906611],
 'V2': [-0.906611],
 'V3': [-0.906611],
 'V4': [-0.906611],
 'V5': [-0.906611],
 'V6': [-0.906611],
 'V7': [-0.906611],
 'V8': [-0.906611],
 'V9': [-0.906611],
 'V10': [-0.906611],
 'V11': [-0.906611],
 'V12': [-0.906611],
 'V13': [-0.906611],
 'V14': [-0.906611],
 'V15': [-0.906611],
 'V16': [-0.906611],
 'V17': [-0.906611],
 'V18': [-0.906611],
 'V19': [-0.906611],
 'V20': [-0.906611],
 'V21': [-0.906611],
 'V22': [-0.906611],
 'V23': [-0.906611],
 'V24': [-0.906611],
 'V25': [-0.906611],
 'V26': [-0.906611],
 'Amount': [15.99],
 'userid': 'user99'}

In [36]:
from flask import Flask
from src.prediction_cf.main import predict

app = Flask('test')
ctx = app.test_request_context(json=test_instance)
request = ctx.request

pred_retval = predict(request)
pred_retval

ERROR:root:Raw features: {'Amount': [15.99], 'V1': [-0.906611], 'V10': [-0.906611], 'V11': [-0.906611], 'V12': [-0.906611], 'V13': [-0.906611], 'V14': [-0.906611], 'V15': [-0.906611], 'V16': [-0.906611], 'V17': [-0.906611], 'V18': [-0.906611], 'V19': [-0.906611], 'V2': [-0.906611], 'V20': [-0.906611], 'V21': [-0.906611], 'V22': [-0.906611], 'V23': [-0.906611], 'V24': [-0.906611], 'V25': [-0.906611], 'V26': [-0.906611], 'V3': [-0.906611], 'V4': [-0.906611], 'V5': [-0.906611], 'V6': [-0.906611], 'V7': [-0.906611], 'V8': [-0.906611], 'V9': [-0.906611], 'userid': 'user99'}


Raw features: {'Amount': [15.99], 'V1': [-0.906611], 'V10': [-0.906611], 'V11': [-0.906611], 'V12': [-0.906611], 'V13': [-0.906611], 'V14': [-0.906611], 'V15': [-0.906611], 'V16': [-0.906611], 'V17': [-0.906611], 'V18': [-0.906611], 'V19': [-0.906611], 'V2': [-0.906611], 'V20': [-0.906611], 'V21': [-0.906611], 'V22': [-0.906611], 'V23': [-0.906611], 'V24': [-0.906611], 'V25': [-0.906611], 'V26': [-0.906611], 'V3': [-0.906611], 'V4': [-0.906611], 'V5': [-0.906611], 'V6': [-0.906611], 'V7': [-0.906611], 'V8': [-0.906611], 'V9': [-0.906611], 'userid': 'user99'}


DEBUG:google.auth._default:Checking None for explicit credentials as part of auth process...


Checking None for explicit credentials as part of auth process...


DEBUG:google.auth._default:Checking Cloud SDK credentials as part of auth process...


...

DEBUG:urllib3.connectionpool:http://metadata.google.internal:80 "GET /computeMetadata/v1/instance/service-accounts/122302747547-compute@developer.gserviceaccount.com/token?scopes=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform HTTP/1.1" 200 1083


http://metadata.google.internal:80 "GET /computeMetadata/v1/instance/service-accounts/122302747547-compute@developer.gserviceaccount.com/token?scopes=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform HTTP/1.1" 200 1083


ERROR:root:Prediction: {'classes': ['legit', 'fraudulent'], 'scores': [0.500018299, 0.499981701]}


Prediction: {'classes': ['legit', 'fraudulent'], 'scores': [0.500018299, 0.499981701]}


'{"classes": ["legit", "fraudulent"], "scores": [0.500018299, 0.499981701]}'

In [37]:
GOOGLE_FUNCTION_SOURCE ='src/prediction_cf/main.py'

ENV_VARS=f"""\
PROJECT={PROJECT},\
REGION={REGION},\
ENDPOINT_NAME={endpoint.name},\
ENTITY={entity},\
FEATURESTORE_ID={FEATURESTORE_ID}
"""

!echo {ENV_VARS}

PROJECT=vertex-mlops-new,REGION=us-central1,ENDPOINT_NAME=8324580654775795712,ENTITY=user,FEATURESTORE_ID=creditcards


In [38]:
!rm -rf src/prediction_cf/.ipynb_checkpoints

In [39]:
!echo gcloud functions deploy {PREDICT_CLOUD_FUNCTION_NAME} --gen2 \
    --set-build-env-vars=GOOGLE_FUNCTION_SOURCE={GOOGLE_FUNCTION_SOURCE} \
    --region={CF_REGION} \
    --runtime=python38 \
    --trigger-http \
    --source=. \
    --entry-point=predict\
    --stage-bucket={BUCKET}\
    --ingress-settings=internal-only\
    --service-account={SERVICE_ACCOUNT}\
    --set-env-vars={ENV_VARS}      

gcloud functions deploy predict-creditcards-classifier-v02-train-pipeline-fn --gen2 --set-build-env-vars=GOOGLE_FUNCTION_SOURCE=src/prediction_cf/main.py --region= --runtime=python38 --trigger-http --source=. --entry-point=predict --stage-bucket=vertex-mlops-new --ingress-settings=internal-only --service-account=122302747547-compute@developer.gserviceaccount.com --set-env-vars=PROJECT=vertex-mlops-new,REGION=us-central1,ENDPOINT_NAME=8324580654775795712,ENTITY=user,FEATURESTORE_ID=creditcards


#### Test the prediction cloud function

You can test it using a `curl` command, but this has to be executed from the same VPC that the Cloud Function is deployed in:

```
curl -m 70 -X POST https://PROJECT.cloudfunctions.net/predict-creditcards-classifier-v02-train-pipeline-fn \
-H "Authorization:bearer $(gcloud auth print-identity-token)" \
-H "Content-Type:application/json" \
-d '{"V1": [-0.906611], "V2": [-0.906611], "V3": [-0.906611], "V4": [-0.906611], "V5": [-0.906611], "V6": [-0.906611], "V7": [-0.906611], "V8": [-0.906611], "V9": [-0.906611], "V10": [-0.906611], "V11": [-0.906611], "V12": [-0.906611], "V13": [-0.906611], "V14": [-0.906611], "V15": [-0.906611], "V16": [-0.906611], "V17": [-0.906611], "V18": [-0.906611], "V19": [-0.906611], "V20": [-0.906611], "V21": [-0.906611], "V22": [-0.906611], "V23": [-0.906611], "V24": [-0.906611], "V25": [-0.906611], "V26": [-0.906611], "Amount": [15.99], "userid": "user99"}'
```

You deploy a VM in the same VPC and run the command from there.

Perhaps an easier way to test is test it from the Testing tab on the [web Console](https://console.cloud.google.com/functions/list).

## Conclusion

We have defined a Vertex AI Pipeline to train a model using TFX. The model uses credit card transaction data where some of the transactions are labeled fraudulent and it predicts whethers transactions are fraudulent.

We have defined and deployed a continuous training pipeline, allowing the model to be retrained by sending a message to a Pub/Sub topic.

We have deployed a Feature Store and loaded it with data. We have deployed a Vertex AI Endpoint to host the model.

We have defined and deployed a model deployment pipeline, that tests the trained model, deploys it to the Endpoint nad tests the Endpoint.

Finally we have deployed a Cloud Function that can serve as the final prediction API, which retrieves some features from the Feature Store, uses these to feed the model and returns the prediction.